In [22]:
# Test generated by ChatGPT

In [23]:
# !pip install ecos

In [15]:
# Charging Network Operator optimization problem

import numpy as np
import cvxpy as cp
import pandas as pd
import networkx as nx

n_nodes = 5
edges = [(0, 1, 6), (1, 2, 6), (2, 3, 8), (3, 4, 12)]  # Linear graph

G = nx.DiGraph()
for u, v, weight in edges:
    G.add_edge(u, v, weight=weight)

# Parameters
maintenance_cost = np.array([1000, 0.2, 0.2, 0.2, 0.2]) # THis is a very important factor
base_demand = np.array([3, 15, 18, 20, 7]) # Demand for charging at each node
MAX_ENERGY = 60  # Full battery
MIN_SOC = 12  # Min SOC required at destination. 20% SOC
initial_SOC = 18  # Start at node 0

p = cp.Variable(n_nodes)                # Price at each node
b = cp.Variable(n_nodes, boolean=True)  # binary selector for price


constraints = [
    p >= 0, # Electricity price is positive
    cp.sum(b) == 2 # Ensure 2 charging stations in the network. Causing problems in our optimization!
]

# Constrain price to either 0 or 0.15
for i in range(n_nodes):
    constraints.append(p[i] == (0.15 * b[i])) # Should I multiply by x[i] to make sure only nodes with station have price

# Energy tracking across the path
soc = initial_SOC
for u, v, weight in edges:
    edge_energy = G[u][v]['weight']
    soc -= edge_energy
    if soc < 0:
        raise ValueError("EV cannot reach destination even with chargers")
    if soc < MIN_SOC: # Place charging station if soc falls below 12
        # Must be able to recharge at node u before continuing
        constraints.append(b[u] >= 1) # Also should be able to charge before node u?

        soc = MAX_ENERGY  # Recharge to full (simplified)


revenue = cp.multiply(p, base_demand)
total_revenue = cp.sum(revenue)
total_cost = maintenance_cost @ b
profit = total_revenue - total_cost
reg = 10 * cp.sum_squares(p) # Manually tuned to get one charger for price of 0.15

objective = cp.Maximize(profit - reg)
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS_BB)


result = pd.DataFrame({
    "Node": np.arange(n_nodes),
    "Charger Installed": (b.value >= 0.5).astype(int),
    "Price": np.round(p.value, 2),
    "Maint Cost": maintenance_cost
})

print(result)
print("Total Profit: ", profit.value)

   Node  Charger Installed  Price  Maint Cost
0     0                  0  -0.00      1000.0
1     1                  1   0.15         0.2
2     2                  0   0.00         0.2
3     3                  1   0.15         0.2
4     4                  0   0.00         0.2
Total Profit:  4.849999999940185


In [ ]:
# Vehicle optimization problem

# ------ Using temporal logic and contracts formats, not just straight up constraining ------ #
"""
# Contract implementation Pseudo code according to ChatGPT #

contract = {
    "A": "SOC_start >= 18", # Start SOC assumption
    "G": "SOC_end >= 12", # End SOC guarantee
    "orig_G": "SOC_end >= 12"
}

# You would enforce contract['G'] as a constraint
assert SOC[0] >= 18       # Assumption A
constraints.append(SOC[-1] >= 12)  # Guarantee G

# Generate MIP constraints like
enforce_formula(add_formula(contract['orig_G']))

# Check if the topology selected allows the contract to be fulfilled
is_refined = check_refine({
    "A": "SOC_start >= 18",
    "G": "SOC_end >= 12",
    "orig_G": "SOC_end >= 12"
}, current_topology_contract, encoding_style="suffi_and_neces")
"""

# ------ Using temporal logic and contracts formats, not just straight up constraining ------ #


c = cp.variable(5) # Represents state of charge at each node
b = cp.variable(5) # Indicator of whether individual charged at node


constraints = [
    c[0] == 0.3, # Initial charge of 18 kWh. Could initialize randomly and see how results change
    c[-1] >= 0.3, # End charge of at least 18 kWh
    cp.sum(b) >= 1 # Must stop at one or more charging station
]



In [ ]:
# # Linearly connected graph

# import numpy as np
# import cvxpy as cp
# import pandas as pd
# import networkx as nx

# n_nodes = 5
# edges = [(0, 1, 0), (1, 2, 6), (2, 3, 8), (3, 4, 12)]  # Linear graph

# G = nx.DiGraph()
# for u, v, weight in edges:
#     G.add_edge(u, v, weight=weight)

# # Parameters
# # budget = 8
# # development_cost = np.array([1_000, 2.0, 2.0, 1.5, 2.0])
# maintenance_cost = np.array([0.25, 0.2, 0.2, 0.2, 0.2]) # THis is a very important factor
# base_demand = np.array([3, 15, 18, 20, 7]) # Demand for charging at each node
# # alpha = 0.05  # Price sensitivity
# MAX_ENERGY = 60  # Full battery
# MIN_SOC = 18  # Min SOC required at destination
# initial_SOC = 18  # Start at node 0

# x = cp.Variable(n_nodes, boolean=True)  # Charger placement
# p = cp.Variable(n_nodes)                # Price at each node
# b = cp.Variable(n_nodes, boolean=True)  # binary selector for price


# constraints = [
#     # cp.sum(cp.multiply(development_cost, x)) <= budget,
#     # p <= 2 * x,
#     p >= 0, # Electricity price is positive
#     p <= x,

#     # 1 - alpha * p >= 0,
#     cp.sum(x) == 2 # Ensure 2 charging stations in the network,
#     ]

# # Constrain price to either 0.2 or 0.5
# for i in range(n_nodes):
#     constraints.append(p[i] == (0.2 * (1 - b[i]) + 0.15 * b[i])) # Should I multiply by x[i] to make sure only nodes with station have price


# # Energy tracking across the path
# soc = initial_SOC
# for u, v, weight in edges:
#     edge_energy = G[u][v]['weight']
#     soc -= edge_energy
#     if soc < 0:
#         raise ValueError("EV cannot reach destination even with chargers")
#     if soc < MIN_SOC:
#         # Must be able to recharge at node u before continuing
#         constraints.append(x[u] >= 1) # Also should be able to charge before node u?
#         soc = MAX_ENERGY  # Recharge to full (simplified)


# # demand = cp.multiply(base_demand, 1 - alpha * p) # Non-convex!!!
# revenue = cp.multiply(p, base_demand)
# # total_revenue = cp.sum(cp.multiply(revenue, x))
# total_revenue = cp.sum(revenue)
# # total_cost = development_cost @ x + maintenance_cost @ x
# total_cost = maintenance_cost @ x
# profit = total_revenue - total_cost
# # profit = total_revenue
# reg = 10 * cp.sum_squares(p) # Manually tuned to get one charger for price of 0.15

# objective = cp.Maximize(profit - reg)
# problem = cp.Problem(objective, constraints)
# problem.solve(solver=cp.ECOS_BB)


# result = pd.DataFrame({
#     "Node": np.arange(n_nodes),
#     # "Charger Installed": (x.value >= 0.5).astype(int),
#     "Price": np.round(p.value, 2),
#     "Maint Cost": maintenance_cost
# })

# print(result)
# print("Total Profit: ", profit.value)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [31]:

p.value[x.value < 0.01] = 0 
p.value

array([0. , 0.2, 0.2, 0. , 0. ])

In [ ]:
p.value @ base_demand

6.599999999172775

In [30]:
# # Densely connected graph

# import numpy as np
# import cvxpy as cp
# import networkx as nx
# import pandas as pd

# # Define graph and parameters
# n_nodes = 6

# # Define edges = (u,v,weight)
# edges = [
#     (0, 1, 10), (1, 2, 8), (2, 3, 6),
#     (0, 4, 15), (4, 5, 5), (5, 3, 10),
#     (1, 5, 4), (2, 4, 7)
# ]
# G = nx.DiGraph()
# for u, v, weight in edges:
#     G.add_edge(u, v, weight=weight)

# ports = [0, 3]
# budget = 3
# development_cost = np.array([2, 1.5, 1, 1.5, 2, 2])
# maintenance_cost = np.array([0.2, 0.1, 0.1, 0.2, 0.2, 0.1])
# price_penalty = np.array([0.5, 0.3, 0.4, 0.6, 0.5, 0.4])

# x = cp.Variable(n_nodes, boolean=True) # Random variable for placement of chargers
# p = cp.Variable(n_nodes) # Random variable for electricity price

# # Find paths and build constraints
# paths = []
# for i in ports:
#     for j in ports:
#         if i != j:
#             try:
#                 path = nx.shortest_path(G, source=i, target=j, weight='weight')
#                 energy_required = sum(G[u][v]['weight'] for u, v in zip(path[:-1], path[1:]))
#                 paths.append((i, j, path, energy_required))
#             except nx.NetworkXNoPath:
#                 continue

# constraints = [cp.sum(cp.multiply(development_cost, x)) <= budget]
# # for (i, j, path, energy_required) in paths:
# #     constraints.append(cp.sum(x[path]) * 60 >= energy_required - 36)

# # Add max distance between chargers constraint (18 kWh segments)
# MAX_DISTANCE = 18

# # How do we encode profit maximization of CNO?

# for (i, j, path, _) in paths:
#     for start in range(len(path)):
#         total_energy = 0
#         for end in range(start + 1, len(path)):
#             u, v = path[end - 1], path[end]
#             total_energy += G[u][v]['weight']
#             if total_energy > MAX_DISTANCE:
#                 subpath = path[start:end]
#                 # At least one internal node must have a charger
#                 internal_nodes = subpath[1:-1]
#                 if internal_nodes:
#                     constraints.append(cp.sum(x[internal_nodes]) >= 1)


# # Encorporate variable p to encourage CNO to maximize profits. Also add regularization to prevent p from overpowering.
# objective = cp.Minimize(-p + price_penalty @ x + maintenance_cost @ x + 0.1 * cp.sum(x))
# problem = cp.Problem(objective, constraints)
# problem.solve()


# # Output results
# result = pd.DataFrame({
#     "Node": np.arange(n_nodes),
#     "Charger Installed": (x.value >= 0.5).astype(int),
#     "Development Cost": development_cost,
#     "Maintenance Cost": maintenance_cost,
#     "Price Penalty": price_penalty
# })

# print(result)


